# 1. Energy data

## 1.0 Startup

In [23]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [24]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [25]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel', 'models','t_dGP_train')

In [26]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel');
models = s.models;


In [31]:
criterions = {'BCM','PoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans BCM
BCM: SMSE 0.0015, MSLL -3.5899, NLPD -0.7299
Processing energy_data kmeans PoE
PoE: SMSE 0.0019, MSLL -3.2017, NLPD -0.3417


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [4]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans RBCM
Linesearch     25;  Value -5.358045e+04
RBCM: SMSE 0.0016, MSLL 4.0186, NLPD 6.8912


### 1.1.2 Grid q

In [5]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
TERBCM: SMSE 0.0018, MSLL 7.1690, NLPD 10.0416 - q   0.20
TERBCM: SMSE 0.0018, MSLL 3.9061, NLPD 6.7787 - q   0.40
TERBCM: SMSE 0.0017, MSLL 2.9821, NLPD 5.8547 - q   0.60
TERBCM: SMSE 0.0017, MSLL 3.0714, NLPD 5.9441 - q   0.80
TERBCM: SMSE 0.0016, MSLL 4.0186, NLPD 6.8912 - q   1.00
TERBCM: SMSE 0.0015, MSLL 6.0415, NLPD 8.9141 - q   1.20
TERBCM: SMSE 0.0015, MSLL 9.6977, NLPD 12.5703 - q   1.40
TERBCM: SMSE 0.0014, MSLL 16.0539, NLPD 18.9266 - q   1.60
TERBCM: SMSE 0.0014, MSLL 27.0656, NLPD 29.9383 - q   1.80
TERBCM: SMSE 0.0013, MSLL 46.3146, NLPD 49.1872 - q   2.00
TERBCM: SMSE 0.0013, MSLL 80.4234, NLPD 83.2961 - q   2.20
TERBCM: SMSE 0.0012, MSLL 141.7737, NLPD 144.6464 - q   2.40
TERBCM: SMSE 0.0012, MSLL 253.7664, NLPD 256.6390 - q   2.60
TERBCM: SMSE 0.0012, MSLL 461.0706, NLPD 463.9433 - q   2.80
TERBCM: SMSE 0.0012, MSLL 849.7301, NLPD 852.6027 - q   3.00


### 1.1.3 Find single q

In [11]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=150, bs=100, lr=0.0001, lambda=10, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 150, q - 1.5000
grad_q_norm =   -9.9509e+06
grad_q_reg =   -1.1169e+05
ttb =    3.0519e+06
Optimizing q: remain iter - 149, q - 1.2741
grad_q_norm =   -6.7268e+06
grad_q_reg =   -3.5587e+04
ttb =    1.2343e+06
Optimizing q: remain iter - 148, q - 1.1296
grad_q_norm =   -8.8799e+06
grad_q_reg = -4533.7
ttb =    4.2798e+05
Optimizing q: remain iter - 147, q - 0.9474
grad_q_norm =   -1.3945e+06
grad_q_reg = -7353.5
ttb =    1.2175e+06
Optimizing q: remain iter - 146, q - 0.9175
grad_q_norm =   -8.8443e+06
grad_q_reg = -8835.7
ttb =    1.3457e+06
Optimizing q: remain iter - 145, q - 0.7352
grad_q_norm =    2.9963e+04
grad_q_reg =  4955.6
ttb =    6.9672e+05
Optimizing q: remain iter - 144, q - 0.7368
grad_q_norm =    3.6582e+05
grad_q_reg =  6985.2
ttb =    9.8838e+05
Optimizing q: remain iter - 143, q - 0.7457
grad_q_norm =    1.3706e+05
grad_q_reg =  7312.2
ttb =    1.0164e+06
Optimizing q: remain iter - 142, q - 0.7500
grad_q_norm =   -6.7149e+07

In [12]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q=0.7247) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.1.4 Find multi q

In [15]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.001, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.8141
   1.3987
   1.7297
   1.1139
   1.7243
   1.4425
   1.7310
   1.9814
   1.5509
   1.3733
   1.1265
   1.7815
   1.8431
   1.7216
   1.3133
   1.9857
   1.4551
   1.1033
   1.0580
   1.6486
   1.5567
   1.3069
   1.7618
   1.6833
   1.4176
   1.1148
   1.0330
   1.4265
   1.3276
   1.6306
   1.9111
   1.1826
   1.9234
   1.3356
   1.9088
   1.2475
   1.5274
   1.2154
   1.2155
   1.8582
   1.2514
   1.5468
   1.7489
   1.7432
   1.3697
   1.5024
   1.7319
   1.1804
   1.4973
   1.6607

ttb =    3.9248e+06
Optimizing qs: remain iter - 49
qs =

   1.66218
   1.34402
   1.72921
   1.11666
   1.71985
   1.08354
   1.71178
   1.72324
   1.55096
   1.34672
   1.12802
   1.18606
   1.76468
   1.71816
   1.31270
   1.71596
   1.38625
   1.10583
   1.03575
   1.64338
   1.55430
   1.30038
   1.74258
   1.68368
   0.79894
   1.10767
   1.03298
   1.42293
   1.25814
   1.61497
   1.29952
   1.10520
   1.76539
   1.33582
   1.90893
   1.

   0.58743
   0.37553
   0.50078
   0.30375
   0.66788
   0.35298
   0.59249
   0.67257
   0.30333
   0.47974
   0.46989

ttb =    1.7788e+06
Optimizing qs: remain iter - 23
qs =

   1.16347
   0.95038
   0.51390
   0.77607
   0.47001
   0.77323
   0.45134
   0.38256
   0.73177
   0.60660
   0.92138
   0.27036
   0.67342
   0.69724
   0.60840
   0.62952
   0.40720
   0.88704
   0.52351
   0.55871
   1.16134
   0.27552
   0.53673
   0.93758
   0.67727
   0.64134
   0.30085
   0.69154
   0.48819
   0.70751
   0.51447
   0.75333
   0.30322
   0.36090
   1.19818
   0.36194
   0.71770
   0.30583
   0.41298
   0.59202
   0.38657
   0.50258
   0.32923
   0.66871
   0.36263
   0.60638
   0.66157
   0.36658
   0.49126
   0.48626

ttb =    6.7295e+05
Optimizing qs: remain iter - 22
qs =

   1.17344
   0.94915
   0.51371
   0.77899
   0.49503
   0.77467
   0.47652
   0.37897
   0.72633
   0.59631
   0.92365
   0.27089
   0.69329
   0.69562
   0.61763
   0.63945
   0.40395
   0.88852
   0.52669
  

In [16]:
qs
save('./res/terbcmenergydataqs', 'qs');
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.00420
   0.78930
   0.56419
   0.78534
   0.68780
   0.57549
   0.70141
   0.69640
   0.73483
   0.60860
   0.90667
   0.45793
   0.77591
   0.72160
   0.71600
   0.76833
   0.58251
   0.85769
   0.66873
   0.63632
   1.08460
   0.63993
   0.69438
   0.94057
   0.70037
   0.68209
   0.48871
   0.68533
   0.72309
   0.71174
   0.74696
   0.70669
   0.57072
   0.62068
   0.87193
   0.65323
   0.69850
   0.58732
   0.80402
   0.63847
   0.57219
   0.57612
   0.54255
   0.66692
   0.41173
   0.65708
   0.60222
   0.59658
   0.65689
   0.64267

TERBCM: SMSE 0.0017, MSLL 3.1604, NLPD 6.0330


In [10]:
save('./res/terbcmenergydataqs_res', 'SMSE','MSLL','NLPD')

## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [5]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GRBCM
GRBCM: SMSE 0.0016, MSLL -4.3158, NLPD -1.4686


### 1.2.2 Grid q

In [6]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGRBCM
TEGRBCM: SMSE 0.0022, MSLL -4.4757, NLPD -1.6285 - q   0.20
TEGRBCM: SMSE 0.0020, MSLL -4.5161, NLPD -1.6689 - q   0.40
TEGRBCM: SMSE 0.0018, MSLL -4.5509, NLPD -1.7037 - q   0.60
TEGRBCM: SMSE 0.0016, MSLL -4.5194, NLPD -1.6722 - q   0.80
TEGRBCM: SMSE 0.0016, MSLL -4.3158, NLPD -1.4686 - q   1.00
TEGRBCM: SMSE 0.0015, MSLL -3.7408, NLPD -0.8937 - q   1.20
TEGRBCM: SMSE 0.0016, MSLL -2.4181, NLPD 0.4291 - q   1.40
TEGRBCM: SMSE 0.0016, MSLL 0.3712, NLPD 3.2183 - q   1.60
TEGRBCM: SMSE 0.0017, MSLL 6.0163, NLPD 8.8634 - q   1.80
TEGRBCM: SMSE 0.0018, MSLL 17.2326, NLPD 20.0798 - q   2.00
TEGRBCM: SMSE 0.0018, MSLL 39.3710, NLPD 42.2182 - q   2.20
TEGRBCM: SMSE 0.0018, MSLL 83.0402, NLPD 85.8874 - q   2.40
TEGRBCM: SMSE 0.0018, MSLL 169.3751, NLPD 172.2223 - q   2.60
TEGRBCM: SMSE 0.0018, MSLL 340.6643, NLPD 343.5114 - q   2.80
TEGRBCM: SMSE 0.0018, MSLL 681.8587, NLPD 684.7059 - q   3.00


### 1.2.3 Find single q

In [13]:
criterion = 'TEGRBCM';

In [13]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=50, bs=100, lr=0.001, lambda=10, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 50, q - 1.5000
grad_q_norm =   -3.9069e+05
grad_q_reg = -408.01
Optimizing q: remain iter - 49, q - 1.4194
grad_q_norm =   -3.0055e+04
grad_q_reg = -82.372
Optimizing q: remain iter - 48, q - 1.4131
grad_q_norm =   -6.3006e+05
grad_q_reg = -157.05
Optimizing q: remain iter - 47, q - 1.2842
grad_q_norm =  1652.1
grad_q_reg = -5.1806
Optimizing q: remain iter - 46, q - 1.2846
grad_q_norm =   -8.4319e+05
grad_q_reg = -92.597
Optimizing q: remain iter - 45, q - 1.1123
grad_q_norm =   -1.1123e+05
grad_q_reg = -43.386
Optimizing q: remain iter - 44, q - 1.0895
grad_q_norm =   -1.3691e+04
grad_q_reg = -32.657
Optimizing q: remain iter - 43, q - 1.0866
grad_q_norm =   -3.4702e+04
grad_q_reg = -53.891
Optimizing q: remain iter - 42, q - 1.0794
grad_q_norm =   -8.0658e+04
grad_q_reg = -35.839
Optimizing q: remain iter - 41, q - 1.0629
grad_q_norm =   -3.3202e+05
grad_q_reg = -36.040
Optimizing q: remain iter - 40, q - 0.9951
grad_q_norm =   -1.3941e+05
gr

In [14]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q=0.5420) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [19]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.01, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.2590
   1.8661
   1.9847
   1.9851
   1.0410
   1.9434
   1.7585
   1.7343
   1.5372
   1.3697
   1.0880
   1.4225
   1.2337
   1.5826
   1.1432
   1.7012
   1.1709
   1.0110
   1.0935
   1.4348
   1.3480
   1.3971
   1.3603
   1.4361
   1.8598
   1.5219
   1.4431
   1.6280
   1.3909
   1.4371
   1.1728
   1.3795
   1.4178
   1.9853
   1.8952
   1.3249
   1.7628
   1.4225
   1.0287
   1.2302
   1.7478
   1.1379
   1.5939
   1.2572
   1.1370
   1.4216
   1.0579
   1.7078
   1.5288
   1.6564

ttb =    8.7300e+04
Optimizing qs: remain iter - 49
qs =

   1.2590
   1.8700
   1.9855
   1.9899
   1.0405
   1.9362
   1.7617
   1.7260
   1.5390
   1.3691
   1.0875
   1.4238
   1.2337
   1.5835
   1.1431
   1.7039
   1.1353
   1.0109
   1.0937
   1.4366
   1.3485
   1.3973
   1.3588
   1.4374
   1.8619
   1.5221
   1.4418
   1.6280
   1.3911
   1.4370
   1.1549
   1.3718
   1.4183
   1.9865
   1.8948
   1.3250
   1.7525
   1.4223
   1.0287


   1.10598
   0.88818
   0.77361
   1.21705
   1.15523
   1.23074
   0.93832
   1.28458
   1.60239
   1.44568
   1.14472
   1.45712
   1.11407
   1.22560
   0.94095
   1.14218
   1.04733
   1.89322
   1.40554
   1.27229
   1.49972
   0.74504
   0.98942
   1.01632
   1.63401
   1.02048
   1.57696
   0.95985
   1.09001
   1.43735
   1.04200
   0.75533
   1.55244
   1.32450

ttb =    2.2627e+04
Optimizing qs: remain iter - 22
qs =

   1.25899
   1.24621
   1.42835
   1.52822
   0.85329
   1.36009
   1.38056
   1.27588
   1.38927
   1.09409
   0.86248
   1.30465
   1.13887
   1.48501
   1.11727
   1.58655
   1.10636
   0.88788
   0.75256
   1.20542
   1.15319
   1.22847
   0.93317
   1.28526
   1.60425
   1.44307
   1.10189
   1.42611
   1.11333
   1.22363
   0.94068
   1.13943
   1.04474
   1.88037
   1.38918
   1.27099
   1.47754
   0.74196
   0.98242
   0.99141
   1.61164
   1.01895
   1.56767
   0.95311
   1.08801
   1.43868
   1.04227
   0.75421
   1.55485
   1.32093

ttb =    2.7154e

In [20]:
qs
save('./res/tegrbcmenergydataqs', 'qs');
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.25899
   1.05468
   1.21911
   1.31987
   0.78089
   1.31089
   1.14537
   1.08358
   1.32420
   0.49364
   0.40822
   1.17627
   0.34615
   1.42733
   1.02875
   1.27060
   0.97238
   0.87514
   0.73713
   1.02212
   1.00841
   1.18847
   0.87979
   1.18271
   1.44052
   1.36627
   0.69537
   0.85110
   0.96844
   0.39773
   0.84612
   0.98663
   0.83117
   1.59592
   0.54444
   1.25975
   0.83361
   0.72956
   0.84633
   0.89932
   0.71956
   0.99467
   1.43302
   0.79277
   0.88667
   1.43467
   1.00101
   0.69924
   1.57635
   1.07412

TEGRBCM: SMSE 0.1227, MSLL -4.8169, NLPD -1.9698


In [21]:
save('./res/tegrbcmenergydataqs_res', 'SMSE','MSLL','NLPD')

## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [18]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GPoE
GPoE: SMSE 0.0013, MSLL 1.4199, NLPD 4.2867


### 1.3.2 Grid q

In [19]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGPoE
TEGPoE: SMSE 0.0016, MSLL 3.4733, NLPD 6.3401 - q   0.20
TEGPoE: SMSE 0.0015, MSLL 1.2277, NLPD 4.0945 - q   0.40
TEGPoE: SMSE 0.0014, MSLL 0.6154, NLPD 3.4822 - q   0.60
TEGPoE: SMSE 0.0014, MSLL 0.7121, NLPD 3.5790 - q   0.80
TEGPoE: SMSE 0.0013, MSLL 1.4199, NLPD 4.2867 - q   1.00
TEGPoE: SMSE 0.0012, MSLL 2.9083, NLPD 5.7751 - q   1.20
TEGPoE: SMSE 0.0011, MSLL 5.5955, NLPD 8.4623 - q   1.40
TEGPoE: SMSE 0.0011, MSLL 10.2757, NLPD 13.1425 - q   1.60
TEGPoE: SMSE 0.0010, MSLL 18.4015, NLPD 21.2683 - q   1.80
TEGPoE: SMSE 0.0010, MSLL 32.6326, NLPD 35.4994 - q   2.00
TEGPoE: SMSE 0.0010, MSLL 57.8887, NLPD 60.7555 - q   2.20
TEGPoE: SMSE 0.0009, MSLL 103.3757, NLPD 106.2426 - q   2.40
TEGPoE: SMSE 0.0009, MSLL 186.5117, NLPD 189.3785 - q   2.60
TEGPoE: SMSE 0.0009, MSLL 340.5884, NLPD 343.4552 - q   2.80
TEGPoE: SMSE 0.0009, MSLL 629.8246, NLPD 632.6914 - q   3.00


### 1.3.3 Find single q

In [27]:
criterion = 'TEGPoE';

In [28]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=150, bs=100, lr=0.001, lambda=10, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 150, q - 1.5000
grad_q_norm =   -5.8649e+05
grad_q_reg = -1228.4
Optimizing q: remain iter - 149, q - 1.3778
grad_q_norm =   -1.0969e+06
grad_q_reg = -920.28
Optimizing q: remain iter - 148, q - 1.1521
grad_q_norm =   -4.8569e+04
grad_q_reg = -266.45
Optimizing q: remain iter - 147, q - 1.1416
grad_q_norm = -190.60
grad_q_reg = -43.168
Optimizing q: remain iter - 146, q - 1.1415
grad_q_norm =   -3.1281e+04
grad_q_reg = -372.57
Optimizing q: remain iter - 145, q - 1.1343
grad_q_norm = -3261.4
grad_q_reg = -37.254
Optimizing q: remain iter - 144, q - 1.1336
grad_q_norm = -5213.9
grad_q_reg = -94.507
Optimizing q: remain iter - 143, q - 1.1323
grad_q_norm =   -5.4671e+04
grad_q_reg = -252.29
Optimizing q: remain iter - 142, q - 1.1207
grad_q_norm =   -2.7060e+04
grad_q_reg = -174.62
Optimizing q: remain iter - 141, q - 1.1148
grad_q_norm =   -5.8845e+05
grad_q_reg = -178.17
Optimizing q: remain iter - 140, q - 0.9943
grad_q_norm =   -2.3261e+05
gra

In [30]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q=0.6541) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [24]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.1, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.2429
   1.8745
   1.7237
   1.8159
   1.1748
   1.6681
   1.3463
   1.7493
   1.6774
   1.8569
   1.4140
   1.0117
   1.2087
   1.7555
   1.5482
   1.3987
   1.1645
   1.7151
   1.3471
   1.8333
   1.9813
   1.5834
   1.7484
   1.8572
   1.9776
   1.3724
   1.8544
   1.2445
   1.4150
   1.3782
   1.6441
   1.8651
   1.7719
   1.3898
   1.0109
   1.9478
   1.5840
   1.2975
   1.8475
   1.1215
   1.8671
   1.5115
   1.6188
   1.8675
   1.9406
   1.5467
   1.1064
   1.7383
   1.3441
   1.6118

ttb = 0
Optimizing qs: remain iter - 49
qs =

   1.2346
   1.8749
   1.7237
   1.8139
   1.1748
   1.6643
   1.3467
   1.7494
   1.6725
   1.8564
   1.4142
   1.0120
   1.2090
   1.7555
   1.5483
   1.3989
   1.1648
   1.7152
   1.3472
   1.8333
   1.9814
   1.5812
   1.7472
   1.8573
   1.9777
   1.3724
   1.8541
   1.2443
   1.4153
   1.3787
   1.6446
   1.8653
   1.7720
   1.3901
   1.0112
   1.9471
   1.5842
   1.2489
   1.8477
   1.1214
  

   1.15489
   0.83422
   0.98407
   1.82707
   1.15527
   1.14315
   1.29133
   0.78687
   1.15198
   1.17553
   1.38507
   1.01627
   1.29439
   0.75540
   1.02751
   1.37985
   1.13231
   0.73406

ttb = 0
Optimizing qs: remain iter - 22
qs =

   0.90004
   1.29466
   0.81375
   0.82454
   1.16676
   1.11887
   0.82929
   0.56346
   1.16311
   0.85327
   0.43380
   0.91814
   0.45670
   0.90081
   1.37055
   0.58200
   0.37362
   0.98661
   1.17397
   0.71835
   0.36474
   1.29580
   0.77759
   1.25612
   0.98916
   1.38404
   0.71202
   0.62301
   1.18396
   1.30543
   0.62821
   0.71841
   1.15575
   0.83490
   0.98407
   1.82187
   1.12386
   1.14257
   1.24384
   0.79441
   1.15270
   1.17574
   1.38529
   1.01579
   1.29472
   0.75644
   1.01555
   1.38047
   1.13135
   0.73162

ttb = 0
Optimizing qs: remain iter - 21
qs =

   0.89432
   1.11355
   0.80601
   0.69998
   1.16875
   1.12305
   0.86159
   0.59477
   1.14258
   0.24887
   0.58769
   0.92403
   0.45787
   0.90784
   1

In [25]:
qs
save('./res/tegpoeenergydataqs', 'qs');
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.77054
   0.86665
   0.82601
   0.66878
   0.54822
   0.43359
   0.66978
   0.69345
   1.08269
   0.57486
   0.29967
   0.92868
   0.50156
   0.88429
   1.37199
   0.67685
   0.83453
   0.98387
   0.59957
   0.54161
   0.71893
   1.19231
   0.82742
   0.79617
   0.84142
   0.34207
   0.69655
   0.66123
   1.03508
   0.98242
   0.70750
   0.67742
   0.99939
   0.85516
   0.46763
   1.11947
   0.93678
   0.59344
   0.79476
   0.63634
   0.95955
   0.78567
   1.27711
   0.28169
   1.13459
   0.79025
   0.99476
   1.36745
   1.04437
   0.74331

TEGPoE: SMSE 0.0014, MSLL 1.0402, NLPD 3.9070


In [26]:
save('./res/tegpoeenergydataqs_res', 'SMSE','MSLL','NLPD')

# 2. Protein

## 2.0 Startup

In [32]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [33]:
dataset = 'protein'
load ('./data/protein.mat');
[ndata, ~] = size(M);
X = M(:,2:9);
Y = M(:,1);
R = randperm(ndata);        
xt = X(R(1:5730),:);
yt = Y(R(1:5730));
R(1:5730) = [];
x = X(R,:);          
y = Y(R);
n = 40000; nt = 5730;   %Ms = 50

dataset = protein


In [34]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;
opts.numOptFC = 25 ;

partitionCriterion = 'kmeans';
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/proteinmodel', 'models','t_dGP_train')

In [35]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/proteinmodel');
models = s.models;


In [36]:
criterions = {'BCM','PoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans BCM
BCM: SMSE 6.4334, MSLL -0.7461, NLPD 2.4872
Processing protein kmeans PoE
PoE: SMSE 29.5573, MSLL 18.0543, NLPD 21.2876


## 2.1 RBCM vs TERBCM

### 2.1.1 RBCM baseline

In [5]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans RBCM
RBCM: SMSE 15.4088, MSLL -0.5472, NLPD 2.6824


### 2.1.2 Grid q

In [6]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing protein kmeans TERBCM
TERBCM: SMSE 14.1947, MSLL 0.2738, NLPD 3.5035 - q   0.20
TERBCM: SMSE 14.2431, MSLL -0.2422, NLPD 2.9874 - q   0.40
TERBCM: SMSE 14.5320, MSLL -0.4294, NLPD 2.8002 - q   0.60
TERBCM: SMSE 14.9335, MSLL -0.5122, NLPD 2.7174 - q   0.80
TERBCM: SMSE 15.4088, MSLL -0.5472, NLPD 2.6824 - q   1.00
TERBCM: SMSE 15.9396, MSLL -0.5565, NLPD 2.6731 - q   1.20
TERBCM: SMSE 16.5144, MSLL -0.5509, NLPD 2.6788 - q   1.40
TERBCM: SMSE 17.1247, MSLL -0.5362, NLPD 2.6934 - q   1.60
TERBCM: SMSE 17.7631, MSLL -0.5162, NLPD 2.7135 - q   1.80
TERBCM: SMSE 18.4232, MSLL -0.4930, NLPD 2.7366 - q   2.00
TERBCM: SMSE 19.0985, MSLL -0.4683, NLPD 2.7614 - q   2.20
TERBCM: SMSE 19.7830, MSLL -0.4429, NLPD 2.7867 - q   2.40
TERBCM: SMSE 20.4709, MSLL -0.4177, NLPD 2.8119 - q   2.60
TERBCM: SMSE 21.1568, MSLL -0.3930, NLPD 2.8366 - q   2.80
TERBCM: SMSE 21.8358, MSLL -0.3693, NLPD 2.8604 - q   3.00


### 2.1.3 Find single q

In [6]:
criterion='TERBCM';

In [8]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=150, bs=100, lr=100, lambda=1.0, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 150, q - 1.5000
grad_q_norm = -1.1332
grad_q_reg =  0.0097858
Optimizing q: remain iter - 149, q - 1.4771
grad_q_norm =  0.21547
grad_q_reg =  0.0029308
Optimizing q: remain iter - 148, q - 1.4815
grad_q_norm =  39.828
grad_q_reg =  0.25327
Optimizing q: remain iter - 147, q - 2.2815
grad_q_norm =  1.5208
grad_q_reg =  0.011371
Optimizing q: remain iter - 146, q - 2.3128
grad_q_norm =  26.888
grad_q_reg =  0.078745
Optimizing q: remain iter - 145, q - 2.8631
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 144, q - 2.8631
grad_q_norm =  14.913
grad_q_reg =  0.027659
Optimizing q: remain iter - 143, q - 3.1681
grad_q_norm =  2.2957
grad_q_reg =  0.0068910
Optimizing q: remain iter - 142, q - 3.2151
grad_q_norm = -0.29233
grad_q_reg =    2.6253e-04
Optimizing q: remain iter - 141, q - 3.2091
grad_q_norm =   -5.9625e-10
grad_q_reg =    2.1237e-19
Optimizing q: remain iter - 140, q - 3.2091
grad_q_norm =  1.8769
grad_q_reg =  0.0043915
Opt

In [9]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q=5.9183) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 2.1.4 Find multi q

In [10]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.001, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.0791
   1.1448
   1.5122
   1.8487
   1.1353
   1.3778
   1.8543
   1.2952
   1.7728
   1.4895
   1.0820
   1.3610
   1.3025
   1.7540
   1.3772
   1.2228
   1.3931
   1.9081
   1.3298
   1.7130
   1.0664
   1.2351
   1.1140
   1.4456
   1.1145
   1.8473
   1.1790
   1.8396
   1.7930
   1.9833
   1.9651
   1.8087
   1.2778
   1.5512
   1.0957
   1.6266
   1.5449
   1.4078
   1.7934
   1.2488
   1.3602
   1.9499
   1.1424
   1.8107
   1.2814
   1.3848
   1.5140
   1.6233
   1.1601
   1.9257

ttb =  121.08
Optimizing qs: remain iter - 49
qs =

   1.0791
   1.1448
   1.5122
   1.8487
   1.1353
   1.3778
   1.8543
   1.2952
   1.7728
   1.4895
   1.0820
   1.3610
   1.3025
   1.7540
   1.3772
   1.2228
   1.3931
   1.9081
   1.3298
   1.7130
   1.0664
   1.2351
   1.1140
   1.4456
   1.1145
   1.8473
   1.1790
   1.8396
   1.7930
   1.9833
   1.9651
   1.8087
   1.2778
   1.5512
   1.0957
   1.6266
   1.5449
   1.4078
   1.7934
   1.2

   1.7930
   1.9833
   1.9651
   1.8087
   1.2778
   1.5512
   1.0957
   1.6266
   1.5449
   1.4078
   1.7934
   1.2488
   1.3602
   1.9499
   1.1424
   1.8107
   1.2814
   1.3848
   1.5140
   1.6233
   1.1601
   1.9257

ttb =  239.44
Optimizing qs: remain iter - 20
qs =

   1.0791
   1.1448
   1.5122
   1.8487
   1.1353
   1.3778
   1.8543
   1.2952
   1.7728
   1.4895
   1.0820
   1.3610
   1.3025
   1.7540
   1.3772
   1.2228
   1.3931
   1.9081
   1.3298
   1.7130
   1.0664
   1.2351
   1.1140
   1.4456
   1.1145
   1.8473
   1.1790
   1.8396
   1.7930
   1.9833
   1.9651
   1.8087
   1.2778
   1.5512
   1.0957
   1.6266
   1.5449
   1.4078
   1.7934
   1.2488
   1.3602
   1.9499
   1.1424
   1.8107
   1.2814
   1.3848
   1.5140
   1.6233
   1.1601
   1.9257

ttb =  235.76
Optimizing qs: remain iter - 19
qs =

   1.0791
   1.1448
   1.5122
   1.8487
   1.1353
   1.3778
   1.8543
   1.2952
   1.7728
   1.4895
   1.0820
   1.3610
   1.3025
   1.7540
   1.3772
   1.2228
   1.3931
   1

In [11]:
qs
save('./res/terbcmproteinqs', 'qs');
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.0791
   1.1448
   1.5122
   1.8487
   1.1353
   1.3778
   1.8543
   1.2952
   1.7728
   1.4895
   1.0820
   1.3610
   1.3025
   1.7540
   1.3772
   1.2228
   1.3931
   1.9081
   1.3298
   1.7130
   1.0664
   1.2351
   1.1140
   1.4456
   1.1145
   1.8473
   1.1790
   1.8396
   1.7930
   1.9833
   1.9651
   1.8087
   1.2778
   1.5512
   1.0957
   1.6266
   1.5449
   1.4078
   1.7934
   1.2488
   1.3602
   1.9499
   1.1424
   1.8107
   1.2814
   1.3848
   1.5140
   1.6233
   1.1601
   1.9257

TERBCM: SMSE 6.7271, MSLL -0.8584, NLPD 2.3794


## 2.2 GRBCM vs TEGRBCM

### 2.2.1 GRBCM baseline

In [6]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans GRBCM
GRBCM: SMSE 5.2694, MSLL -0.9650, NLPD 2.2689


### 2.2.2 Grid q

In [7]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing protein kmeans TEGRBCM
TEGRBCM: SMSE 4.6107, MSLL -1.0649, NLPD 2.1689 - q   0.20
TEGRBCM: SMSE 4.6058, MSLL -1.0838, NLPD 2.1500 - q   0.40
TEGRBCM: SMSE 4.7329, MSLL -1.0524, NLPD 2.1814 - q   0.60
TEGRBCM: SMSE 4.9569, MSLL -1.0100, NLPD 2.2238 - q   0.80
TEGRBCM: SMSE 5.2694, MSLL -0.9650, NLPD 2.2689 - q   1.00
TEGRBCM: SMSE 5.6664, MSLL -0.9198, NLPD 2.3140 - q   1.20
TEGRBCM: SMSE 6.1435, MSLL -0.8758, NLPD 2.3580 - q   1.40
TEGRBCM: SMSE 6.6948, MSLL -0.8335, NLPD 2.4003 - q   1.60
TEGRBCM: SMSE 7.3124, MSLL -0.7932, NLPD 2.4406 - q   1.80
TEGRBCM: SMSE 7.9869, MSLL -0.7551, NLPD 2.4788 - q   2.00
TEGRBCM: SMSE 8.7075, MSLL -0.7191, NLPD 2.5147 - q   2.20
TEGRBCM: SMSE 9.4628, MSLL -0.6854, NLPD 2.5484 - q   2.40
TEGRBCM: SMSE 10.2416, MSLL -0.6540, NLPD 2.5798 - q   2.60
TEGRBCM: SMSE 11.0328, MSLL -0.6247, NLPD 2.6091 - q   2.80
TEGRBCM: SMSE 11.8264, MSLL -0.5975, NLPD 2.6363 - q   3.00


### 2.2.3 Find single q

In [5]:
criterion = 'TEGRBCM';

In [6]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=50, bs=100, lr=100.0, lambda=10, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 50, q - 1.5000
grad_q_norm =  53.885
grad_q_reg =  0.46620
Optimizing q: remain iter - 49, q - 2.3000
grad_q_norm =  32.964
grad_q_reg =  0.15002
Optimizing q: remain iter - 48, q - 3.0033
grad_q_norm =  5.3236
grad_q_reg =  0.030797
Optimizing q: remain iter - 47, q - 3.1183
grad_q_norm =  1.7370
grad_q_reg =  0.024775
Optimizing q: remain iter - 46, q - 3.1588
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 45, q - 3.1588
grad_q_norm =  0.45422
grad_q_reg =  0.015101
Optimizing q: remain iter - 44, q - 3.1711
grad_q_norm =   -5.8959e-19
grad_q_reg =    3.7282e-19
Optimizing q: remain iter - 43, q - 3.1711
grad_q_norm = -0.70385
grad_q_reg =  0.026561
Optimizing q: remain iter - 42, q - 3.1622
grad_q_norm =  6.0080
grad_q_reg =  0.033153
Optimizing q: remain iter - 41, q - 3.2916
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 40, q - 3.2916
grad_q_norm =  0.085125
grad_q_reg =  0.0027382
Optimizing q: remain iter - 39, q 

In [8]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q=4.6433) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 2.2.4 Find multi q

In [29]:
criterion

criterion = TEGRBCM


In [6]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.1, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.2248
   1.9469
   1.9295
   1.2185
   1.6271
   1.7702
   1.6516
   1.9451
   1.4453
   1.5929
   1.3332
   1.9493
   1.4598
   1.2879
   1.1073
   1.4124
   1.0996
   1.7244
   1.7195
   1.9737
   1.1508
   1.9555
   1.0913
   1.6245
   1.6102
   1.5310
   1.1962
   1.3204
   1.8989
   1.5135
   1.6817
   1.5476
   1.8859
   1.2498
   1.6027
   1.5043
   1.1318
   1.2403
   1.9180
   1.3138
   1.5409
   1.9363
   1.0363
   1.2069
   1.6050
   1.0836
   1.7206
   1.3306
   1.4174
   1.9699

ttb =  733.86
Optimizing qs: remain iter - 49
qs =

   1.2248
   1.9469
   1.9295
   1.2185
   1.6271
   1.7702
   1.6516
   1.9451
   1.4453
   1.5929
   1.3332
   1.9493
   1.4598
   1.2879
   1.1073
   1.4124
   1.0996
   1.7244
   1.7195
   1.9737
   1.1508
   1.9555
   1.0913
   1.6245
   1.6102
   1.5310
   1.1962
   1.3204
   1.8989
   1.5135
   1.6817
   1.5476
   1.8859
   1.2498
   1.6027
   1.5043
   1.1318
   1.2403
   1.9180
   1.3

   1.9000
   1.5137
   1.6819
   1.5475
   1.8858
   1.2497
   1.6031
   1.5043
   1.1317
   1.2403
   1.9181
   1.3167
   1.5409
   1.9363
   1.0364
   1.2069
   1.6050
   1.0835
   1.7206
   1.3306
   1.4187
   1.9698

ttb =  770.89
Optimizing qs: remain iter - 20
qs =

   1.2248
   1.9474
   1.9297
   1.2206
   1.6280
   1.7702
   1.6516
   1.9450
   1.4455
   1.5928
   1.3340
   1.9492
   1.4597
   1.2878
   1.1109
   1.4127
   1.1001
   1.7244
   1.7195
   1.9738
   1.1507
   1.9554
   1.0913
   1.6245
   1.6102
   1.5309
   1.1960
   1.3217
   1.9000
   1.5137
   1.6819
   1.5475
   1.8858
   1.2497
   1.6031
   1.5043
   1.1317
   1.2403
   1.9182
   1.3176
   1.5409
   1.9363
   1.0364
   1.2069
   1.6050
   1.0835
   1.7206
   1.3306
   1.4187
   1.9698

ttb =  860.99
Optimizing qs: remain iter - 19
qs =

   1.2248
   1.9474
   1.9297
   1.2206
   1.6280
   1.7702
   1.6516
   1.9450
   1.4455
   1.5928
   1.3340
   1.9492
   1.4597
   1.2878
   1.1110
   1.4127
   1.1001
   1

In [7]:
save qs

In [8]:
qs
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.2248
   1.9475
   1.9296
   1.2205
   1.6279
   1.7702
   1.6516
   1.9450
   1.4457
   1.5928
   1.3341
   1.9492
   1.4597
   1.2878
   1.1109
   1.4130
   1.1001
   1.7243
   1.7195
   1.9738
   1.1507
   1.9554
   1.0912
   1.6245
   1.6104
   1.5309
   1.1960
   1.3217
   1.9003
   1.5147
   1.6819
   1.5475
   1.8858
   1.2497
   1.6031
   1.5043
   1.1316
   1.2402
   1.9186
   1.3203
   1.5409
   1.9363
   1.0369
   1.2069
   1.6050
   1.0835
   1.7206
   1.3306
   1.4186
   1.9698

TEGRBCM: SMSE 6.7513, MSLL -0.8386, NLPD 2.3976


In [13]:
save('./res/tegrbcmenergydataqs_res', 'SMSE','MSLL','NLPD')

## 2.3 GPoE vs TEGPoE

### 2.3.1 GPoE baseline

In [9]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans GPoE
GPoE: SMSE 5.7003, MSLL -0.8517, NLPD 2.3841


### 2.3.2 Grid q

In [5]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing protein kmeans TEGPoE
TEGPoE: SMSE 5.9310, MSLL 0.0062, NLPD 3.2420 - q   0.20
TEGPoE: SMSE 5.8730, MSLL -0.5402, NLPD 2.6956 - q   0.40
TEGPoE: SMSE 5.8152, MSLL -0.7373, NLPD 2.4985 - q   0.60
TEGPoE: SMSE 5.7575, MSLL -0.8212, NLPD 2.4147 - q   0.80
TEGPoE: SMSE 5.7003, MSLL -0.8517, NLPD 2.3841 - q   1.00
TEGPoE: SMSE 5.6437, MSLL -0.8525, NLPD 2.3833 - q   1.20
TEGPoE: SMSE 5.5878, MSLL -0.8351, NLPD 2.4007 - q   1.40
TEGPoE: SMSE 5.5328, MSLL -0.8060, NLPD 2.4299 - q   1.60
TEGPoE: SMSE 5.4789, MSLL -0.7690, NLPD 2.4668 - q   1.80
TEGPoE: SMSE 5.4262, MSLL -0.7268, NLPD 2.5090 - q   2.00
TEGPoE: SMSE 5.3749, MSLL -0.6811, NLPD 2.5547 - q   2.20
TEGPoE: SMSE 5.3250, MSLL -0.6331, NLPD 2.6027 - q   2.40
TEGPoE: SMSE 5.2767, MSLL -0.5837, NLPD 2.6522 - q   2.60
TEGPoE: SMSE 5.2301, MSLL -0.5334, NLPD 2.7024 - q   2.80
TEGPoE: SMSE 5.1852, MSLL -0.4829, NLPD 2.7529 - q   3.00


### 2.3.3 Find single q

In [15]:
criterion = 'TEGPoE';

In [16]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=150, bs=100, lr=10.0, lambda=10, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 150, q - 1.5000
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 149, q - 1.5000
grad_q_norm =  118.55
grad_q_reg =  0.39772
Optimizing q: remain iter - 148, q - 1.7501
grad_q_norm =  0.18025
grad_q_reg =    1.0614e-06
Optimizing q: remain iter - 147, q - 1.7504
grad_q_norm =   -6.3984e-15
grad_q_reg = 0
Optimizing q: remain iter - 146, q - 1.7504
grad_q_norm =  4.1062
grad_q_reg =  0.0052606
Optimizing q: remain iter - 145, q - 1.7589
grad_q_norm =    7.4594e-04
grad_q_reg =    4.9084e-11
Optimizing q: remain iter - 144, q - 1.7589
grad_q_norm =    3.0792e-06
grad_q_reg =    1.8979e-15
Optimizing q: remain iter - 143, q - 1.7589
grad_q_norm =  2.3697
grad_q_reg =  0.0015390
Optimizing q: remain iter - 142, q - 1.7638
grad_q_norm =  29.401
grad_q_reg =  0.094776
Optimizing q: remain iter - 141, q - 1.8257
grad_q_norm =   -4.2357e-15
grad_q_reg = 0
Optimizing q: remain iter - 140, q - 1.8257
grad_q_norm =  9.6793
grad_q_reg =  0.012605


In [18]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q=4.0663) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 2.3.4 Find multi q

In [21]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=1000.0, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.1544
   1.8749
   1.1502
   1.5176
   1.5907
   1.9623
   1.6591
   1.3590
   1.2030
   1.4276
   1.5025
   1.1675
   1.5461
   1.7492
   1.4681
   1.0142
   1.1995
   1.9157
   1.9964
   1.4222
   1.2929
   1.9437
   1.1611
   1.9074
   1.9302
   1.7965
   1.9283
   1.4919
   1.3007
   1.4342
   1.2810
   1.4401
   1.1315
   1.1718
   1.3533
   1.6079
   1.7318
   1.3377
   1.2304
   1.0712
   1.5407
   1.8320
   1.1700
   1.2532
   1.1052
   1.3916
   1.5259
   1.9297
   1.8876
   1.1110

ttb = 0
Optimizing qs: remain iter - 49
qs =

   1.1899
   1.8749
   1.1502
   1.5176
   1.5907
   1.9623
   1.6591
   1.3590
   1.2030
   1.4276
   1.5025
   1.1675
   1.5461
   1.7492
   1.4681
   1.0142
   1.1995
   1.9157
   1.9964
   1.4222
   1.2929
   1.9437
   1.1611
   1.9074
   1.9302
   1.7965
   1.9283
   1.4919
   1.3007
   1.4342
   1.2810
   1.4401
   1.1315
   1.1718
   1.3533
   1.6079
   1.7318
   1.3377
   1.2304
   1.0712
  

   1.5659
   2.7723
   2.6876
   1.1110

ttb = 0
Optimizing qs: remain iter - 20
qs =

   3.8163
   1.8796
   1.3259
   2.5960
   3.1940
   2.8867
   1.6628
   1.3590
   2.0758
   1.4296
   2.5648
   1.1725
   1.5461
   1.7494
   1.4682
   1.9037
   3.6067
   1.9992
   2.1250
   3.0228
   1.2929
   2.3623
   1.2740
   1.9109
   2.1830
   1.8061
   2.1936
   1.4999
   2.9765
   2.2489
   1.3909
   1.4512
   1.4998
   2.5841
   3.7973
   1.6079
   1.7318
   1.3377
   2.5171
   2.9645
   1.5413
   1.8320
   1.1700
   1.2532
   2.5852
   1.4337
   1.5792
   2.7723
   2.6876
   1.1110

ttb = 0
Optimizing qs: remain iter - 19
qs =

   3.9074
   1.8848
   1.3259
   2.5960
   3.1940
   2.8867
   1.6628
   1.3590
   2.0758
   1.4297
   2.5648
   1.1725
   1.5461
   1.7494
   1.4682
   1.9037
   3.6067
   1.9992
   2.1250
   3.0228
   1.2929
   2.3623
   1.2740
   1.9369
   2.1830
   1.8061
   2.1936
   1.4999
   2.9765
   2.2489
   1.3909
   1.4512
   1.5805
   2.5841
   3.7973
   1.6079
   1.7

In [22]:
qs
save('./res/tegpoeenergydataqs', 'qs');
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   5.1164
   1.8848
   2.4945
   2.6740
   3.8522
   2.8998
   1.6628
   1.3590
   2.0758
   1.4676
   3.1446
   2.3404
   1.5461
   1.8295
   1.6642
   1.9412
   3.6364
   2.6235
   2.1313
   3.2902
   1.2929
   3.2791
   1.2740
   2.3133
   2.9838
   1.8127
   2.2276
   1.5940
   3.0897
   2.2581
   2.1913
   1.4734
   1.5886
   2.7281
   3.7976
   1.6079
   2.6559
   1.3377
   2.5206
   2.9659
   1.5413
   1.8320
   1.9914
   1.2532
   2.8058
   1.4808
   1.5792
   2.7793
   2.6877
   1.1110

TEGPoE: SMSE 4.3481, MSLL -0.6174, NLPD 2.6167


In [ ]:
save('./res/tegpoeenergydataqs_res', 'SMSE','MSLL','NLPD')